In [1]:
testlib_path = "/Users/kai/Projects/AlLib/target/testlib.dylib"

In [2]:
from alchemist import *
import numpy as np

als = AlchemistSession()        # Start Alchemist session

als.connect_to_alchemist("0.0.0.0", 24960)

Starting Alchemist session ... ready
Connecting to Alchemist at 0.0.0.0:24960 ...Connected to Alchemist!


In [3]:
als.request_workers(3) 

Requesting 3 workers from Alchemist
Total allocated 3 workers:
    Worker-1 on KaisMacBookPro.local at 0.0.0.0:24961
    Worker-2 on KaisMacBookPro.local at 0.0.0.0:24962
    Worker-3 on KaisMacBookPro.local at 0.0.0.0:24963
Connecting to Alchemist at KaisMacBookPro.local:24961 ...Connected to Alchemist!
Connecting to Alchemist at KaisMacBookPro.local:24962 ...Connected to Alchemist!
Connecting to Alchemist at KaisMacBookPro.local:24963 ...Connected to Alchemist!


In [4]:
TestLib = als.load_library("TestLib", testlib_path)

Library 'TestLib' successfully loaded.


In [5]:
A = np.random.rand(2000,1000)

A_handle = als.send_matrix(matrix=A, print_times=True, layout="VC_STAR")

Sending array info to Alchemist ... done (3.8446e-01s)
Sending array data to Alchemist ... done (2.5355e+00s)

Data transfer times breakdown
  ---------------------------------------------------------------------------------------------------------------
    Worker  |   Serialization time   |       Send time        |      Receive time      |  Deserialization time  
  ---------------------------------------------------------------------------------------------------------------
        1   |       1.6036e-02       |       3.5410e-03       |       6.1989e-05       |       6.9141e-06       
        2   |       5.9960e-03       |       3.0582e-03       |       6.0320e-05       |       4.2915e-06       
        3   |       7.2472e-03       |       3.9928e-03       |       7.0095e-05       |       2.8610e-06       
  ---------------------------------------------------------------------------------------------------------------



In [6]:
rank = 20
S_handle, U_handle, V_handle = TestLib.truncated_svd(A_handle, rank)

List of input parameters:
    A (MATRIX ID) = 1
    rank (INT) = 20
Alchemist started task 'truncated_svd' ... done (6.9877e-01s)


In [7]:
S = als.fetch_matrix(S_handle, print_times=True)

print(np.flipud(S).transpose())

Fetching data for array S from Alchemist ... done (3.4449e-03s)

Data transfer times breakdown for array S
  ---------------------------------------------------------------------------------------------------------------
    Worker  |   Serialization time   |       Send time        |      Receive time      |  Deserialization time  
  ---------------------------------------------------------------------------------------------------------------
        1   |       8.8930e-05       |       1.0300e-03       |       4.2915e-05       |       3.9411e-04       
        2   |       4.6015e-05       |       4.8161e-05       |       4.2915e-05       |       1.1897e-04       
        3   |       4.0054e-05       |       3.1948e-05       |       3.9816e-05       |       1.0705e-04       
  ---------------------------------------------------------------------------------------------------------------

[[707.4330693   21.91546578  21.79424743  21.77359353  21.67895046
   21.57445775  21.53187135  21

In [8]:
V = als.fetch_matrix(V_handle, print_times=True)
U = als.fetch_matrix(U_handle, print_times=True)
S = als.fetch_matrix(S_handle, print_times=True)

Fetching data for array V from Alchemist ... done (5.4564e-02s)

Data transfer times breakdown for array V
  ---------------------------------------------------------------------------------------------------------------
    Worker  |   Serialization time   |       Send time        |      Receive time      |  Deserialization time  
  ---------------------------------------------------------------------------------------------------------------
        1   |       8.4877e-05       |       2.5415e-04       |       1.0395e-04       |       2.5082e-04       
        2   |       2.5988e-05       |       2.9087e-05       |       1.0538e-04       |       2.5582e-04       
        3   |       2.6226e-05       |       2.5988e-05       |       1.0967e-04       |       2.6298e-04       
  ---------------------------------------------------------------------------------------------------------------

Fetching data for array U from Alchemist ... done (8.0449e-02s)

Data transfer times breakdown for

In [9]:
u, s, vh = np.linalg.svd(A, full_matrices=True)

In [10]:
print(np.linalg.norm(abs(V.T) - abs(np.flipud(vh[0:rank,:]))))
print(np.linalg.norm(abs(U) - abs(np.fliplr(u[:,0:rank]))))
print(np.linalg.norm(np.flip(S).T - s[0:rank]))

1.8692535862447696e-12
1.8672534028413392e-12
1.4413688216379792e-13


In [11]:
als.stop()

Closing socket
Closing socket
Closing socket
Closing socket
